In [1]:
import igraph
import numpy as np
import pandas as pd
import pebble_game_algorithm.lib as lib

In [2]:
g = igraph.Graph.Formula('A,B,C,D,E,F,G,H,I,G ->')

In [3]:
print(g)

IGRAPH DN-- 9 0 --
+ attr: name (v)


In [4]:
edges = [
    ('A', 'B'),
    ('A', 'C'),
    ('A', 'D'),
    ('B', 'C'),
    ('B', 'D'),
    ('C', 'D'),
    ('E', 'F'),
    ('E', 'G'),
    ('F', 'G'),
    ('H', 'I'),
]

In [5]:
for v in g.vs:
    v["free_pebble"] = 2

In [6]:
def find_free_pebble(root_vid):
    """Finds a node with a free pebble."""
    path = []
    depth_last = -1
    for (v, depth, _) in g.dfsiter(vid=root_vid, advanced=True, mode="out"):
        if depth != (depth_last + 1):
            path = path[:depth]
        path.append(v)
        depth_last = depth
        if v["free_pebble"] > 0:
            return path
    return None

In [7]:
def transfer_pebble(g, path):
    """Moves a pebble on a path from one end to the other end."""
    start_node = path[0]
    start_node["free_pebble"] += 1
    for node in path[1:]:
        edge = g.get_eid(v1=start_node.index, v2=node.index)
        g.delete_edges(edge)
        g.add_edge(source=node, target=start_node)  # reverse the order
        start_node = node
    start_node["free_pebble"] -= 1

In [8]:
for edge in edges:
    source, target = edge
    independent = True
    free_pebble = {source: 0, target: 0}
    for root in [source, target, source, target]:
        root_node = g.vs.find(root)
        path = find_free_pebble(root_vid=root_node.index)
        if path is None:
            print(f"Edge is redundant: {edge}")
            independent = False
            break
        transfer_pebble(g, path)
        root_node["free_pebble"] -= 1
        free_pebble[root] += 1
    for root in [source, target]:
        g.vs.find(root)["free_pebble"] += free_pebble[root]
    if independent:
        g.add_edge(source=source, target=target)
        g.vs.find(source)["free_pebble"] -=1

Edge is redundant: ('C', 'D')


In [9]:
print(g)

IGRAPH DN-- 9 9 --
+ attr: free_pebble (v), name (v)
+ edges (vertex names):
B->D, A->C, B->C, A->B, D->A, E->G, E->F, F->G, H->I


In [10]:
def find_rigid_cluster(g, edge):
    """Identifies the rigid cluster containing the given edge."""
    source, target = edge
    free_pebble = {source: 0, target: 0}
    for root in [target, target, source]:
        root_node = g.vs.find(root)
        path = find_free_pebble(root_vid=root_node.index)
        if path is None:
            print(f"Inconsistency Eror: no pebble found!")
            break
        transfer_pebble(g, path)
        root_node["free_pebble"] -= 1
        free_pebble[root] += 1
    for root in [source, target]:
        g.vs.find(root)["free_pebble"] += free_pebble[root]

    subgraph = set()
    edges_buffer = []
    g.vs["visited"] = False
    for root in [source, target]:
        root_node = g.vs.find(root)
        for (v, depth, parent) in g.dfsiter(vid=root_node.index, advanced=True, mode="in"):
            if parent is None:
                subgraph.add(v)
                continue
            if v["visited"]:
                subgraph.add(v)
            elif root == source:
                v["visited"] = True
                edges_buffer.append((v["name"], parent["name"]))
        if root == source:
            g.delete_edges(edges_buffer)

    g.add_edges(edges_buffer)

    del g.vs["visited"]
    return subgraph

In [11]:
for edge in edges[:]:
    subgraph = find_rigid_cluster(g, edge)
    print(f"Rigid cluster for edge {edge} is {[v['name'] for v in subgraph]}")

Rigid cluster for edge ('A', 'B') is ['C', 'D', 'A', 'B']
Rigid cluster for edge ('A', 'C') is ['C', 'D', 'A', 'B']
Rigid cluster for edge ('A', 'D') is ['C', 'D', 'A', 'B']
Rigid cluster for edge ('B', 'C') is ['C', 'D', 'A', 'B']
Rigid cluster for edge ('B', 'D') is ['C', 'D', 'A', 'B']
Rigid cluster for edge ('C', 'D') is ['C', 'D', 'A', 'B']
Rigid cluster for edge ('E', 'F') is ['G', 'E', 'F']
Rigid cluster for edge ('E', 'G') is ['G', 'E', 'F']
Rigid cluster for edge ('F', 'G') is ['G', 'E', 'F']
Rigid cluster for edge ('H', 'I') is ['I', 'H']


In [12]:
def find_stressed_cluster(g, edge):
    """Indentifies the stressed cluster containing the given bond."""
    source, target = edge
    free_pebble = {source: 0, target: 0}
    for root in [target, target, source]:
        root_node = g.vs.find(root)
        path = find_free_pebble(root_vid=root_node.index)
        if path is None:
            print(f"Inconsistency Eror: no pebble found!")
            break
        transfer_pebble(g, path)
        root_node["free_pebble"] -= 1
        free_pebble[root] += 1
    for root in [source, target]:
        g.vs.find(root)["free_pebble"] += free_pebble[root]
        
    subgraph = set()
    root_node = g.vs.find(source)
    for v in g.dfsiter(vid=root_node.index, mode="out"):
        subgraph.add(v)

    return subgraph

In [13]:
for edge in edges[:]:
    subgraph = find_stressed_cluster(g, edge)
    print(f"Stressed cluster for edge {edge} is {[v['name'] for v in subgraph]}")

Stressed cluster for edge ('A', 'B') is ['A', 'B']
Stressed cluster for edge ('A', 'C') is ['C', 'A']
Stressed cluster for edge ('A', 'D') is ['D', 'A']
Stressed cluster for edge ('B', 'C') is ['C', 'B']
Stressed cluster for edge ('B', 'D') is ['D', 'B']
Stressed cluster for edge ('C', 'D') is ['C', 'D', 'A', 'B']
Stressed cluster for edge ('E', 'F') is ['E', 'F']
Stressed cluster for edge ('E', 'G') is ['G', 'E']
Stressed cluster for edge ('F', 'G') is ['G', 'F']
Stressed cluster for edge ('H', 'I') is ['I', 'H']
